In [50]:
import onnx
from onnxruntime import InferenceSession
from onnxruntime.transformers.optimizer import optimize_model
from onnxruntime.quantization import quantize_static, CalibrationDataReader, QuantType
import numpy as np

model_path = "model.onnx"
optimized_path = "model_optim.onnx"
quantized_path = "model_quant.onnx"

# === 1. Проверка модели ===
onnx_model = onnx.load(model_path)
onnx.checker.check_model(onnx_model)
print("✅ Исходная модель корректна")

# === 2. Оптимизация графа ===
print("🚀 Оптимизация графа...")
opt_model = optimize_model(model_path, model_type='bert', num_heads=0, hidden_size=0)
opt_model.save_model_to_file(optimized_path)
print(f"✅ Модель оптимизирована и сохранена в {optimized_path}")

# === 3. Калибровочный генератор данных ===
class DummyDataReader(CalibrationDataReader):
    def __init__(self, input_name, num_samples=5, shape=(1, 3, 64, 64)):
        self.input_name = input_name
        self.data = iter([ {input_name: np.random.randn(*shape).astype(np.float32)} for _ in range(num_samples) ])
    def get_next(self):
        return next(self.data, None)

# === 4. Квантизация (статическая, без предупреждений) ===
print("⚙️ Применяем квантизацию...")
sess = InferenceSession(optimized_path)
input_name = sess.get_inputs()[0].name
dr = DummyDataReader(input_name)

quantize_static(
    model_input=optimized_path,
    model_output=quantized_path,
    calibration_data_reader=dr,
    weight_type=QuantType.QInt8,
)
print(f"✅ Квантизованная модель сохранена в {quantized_path}")


✅ Исходная модель корректна
🚀 Оптимизация графа...
✅ Модель оптимизирована и сохранена в model_optim.onnx
⚙️ Применяем квантизацию...


✅ Квантизованная модель сохранена в model_quant.onnx


### Узнаем имена входов и выходов моделей

In [54]:
import onnxruntime
sess = onnxruntime.InferenceSession("./model_repository/model_onnx/1/model.onnx")
print([i.name for i in sess.get_inputs()])
print([o.name for o in sess.get_outputs()])


['input']
['output']


In [55]:
sess = onnxruntime.InferenceSession("./model_repository/model_optim_onnx/1/model_optim.onnx")
print([i.name for i in sess.get_inputs()])
print([o.name for o in sess.get_outputs()])


['input']
['output']
